In [1]:
import numpy as np
from kymograph_synthesis.pipeline import Pipeline
from kymograph_synthesis.params import Params
from kymograph_synthesis.pipeline.steps.sample_kymograph import sample_kymograph
from kymograph_synthesis.pipeline.steps.generate_ground_truth import generate_ground_truth, GroundTruthFuncCollection
from kymograph_synthesis.params.render_params.render_params import (
    _random_relative_particle_path_points,
    _convert_relative_to_um,
)
from kymograph_synthesis.params.ground_truth_params import InstanceGroundTruth, StateGroundTruth

In [ ]:
truth_shape = (32, 32, 768)
truth_scale = (0.04, 0.02, 0.02)
params = Params.model_validate(
    {
        "n_steps": 140,
        "dynamics": [
            {
                "particle_behaviour": "unidirectional",
                "particle_density": 2,
                "antero_speed_mode": 2,
                "antero_speed_var": 0.1**2,
                "retro_speed_mode": 1.5,
                "retro_speed_var": 0.1**2,
                "antero_resample_prob": 0.05,
                "retro_resample_prob": 0,
                "velocity_noise_var": 0.05**2,
                "fluorophore_count_mode": 400,
                "fluorophore_count_var": 40**2,
                "fluorophore_halflife_mode": 140,
                "fluorophore_halflife_var": 14**2,
                "state_ratios": {
                    "anterograde": 1,
                    "stationary": 0,
                    "retrograde": 0,
                },
                "seed": 42
            },
            {
                "particle_behaviour": "unidirectional",
                "particle_density": 2,
                "antero_speed_mode": 2,
                "antero_speed_var": 0.1**2,
                "retro_speed_mode": 1.5,
                "retro_speed_var": 0.1**2,
                "antero_resample_prob": 0.05,
                "retro_resample_prob": 0,
                "velocity_noise_var": 0.05**2,
                "fluorophore_count_mode": 200,
                "fluorophore_count_var": 20**2,
                "fluorophore_halflife_mode": 140,
                "fluorophore_halflife_var": 14**2,
                "state_ratios": {
                    "anterograde": 0,
                    "stationary": 0,
                    "retrograde": 1,
                },
                "seed": 42
            },
        ],
        "rendering": {
            "static_distributions": [
                {
                    "name": "simplex_noise",
                    "max_fluorophore_count_per_nm3": 0.01,
                    "noise_scales": [
                        0.5,
                        1.5,
                    ],
                    "scale_weights": [0.5, 0.5],
                    "seed": 42
                }
            ],
            "particle_path_points": _convert_relative_to_um(
                _random_relative_particle_path_points(5),
                truth_shape,
                truth_scale,
            ),
            "imaging": {
                "exposure_ms": 100,
                "truth_space": {
                    "shape": truth_shape,
                    "scale": truth_scale,
                },
                "detector": {
                    "camera_type": "CCD",
                    "read_noise": 6,
                    "gain": 10,
                    "offset": 100,
                },
                "objective_lens": {"na": 1.4},
                "settings": {"random_seed": 42}#, "np_backend": "cupy"}
            },
        },
        "kymograph": {"n_spatial_values_factor": 1},
    }
)

In [3]:
pipeline = Pipeline(params, out_dir=None)

In [4]:
pipeline.run()

2025-11-28 17:27:43,491 INFO: Creating digital_image ...
2025-11-28 17:27:43,491 INFO: Creating optical_image ...
2025-11-28 17:27:43,492 INFO: Creating ground truth (shape =(32, 32, 768)) + 
/home/melisande.croft/Documents/Repos/kymograph-synthesis/.venv/lib/python3.11/site-packages/cupy/_environment.py:379: UserWarning: CuPy failed to preload library (/home/melisande.croft/.cupy/cuda_lib/12.x/cutensor/2.1.0/lib/libcutensor.so.2.1.0): OSError (libcublasLt.so.12: cannot open shared object file: No such file or directory)
  warnings.warn(msg)


ImportError: libcublas.so.12: cannot open shared object file: No such file or directory

In [ ]:
frames = pipeline.imaging_sim_output['frames']

In [ ]:
mean_proj_kymo = sample_kymograph(params.kymograph, frames, "mean")["kymograph"]
max_proj_kymo = sample_kymograph(params.kymograph, frames, "max")["kymograph"]
slice_proj_kymo_data = sample_kymograph(params.kymograph, frames, "centre_slice")
slice_proj_kymo = slice_proj_kymo_data["kymograph"]

In [ ]:
def create_instance_map(instance_gt):
    instance_map = np.zeros(instance_gt.shape[:2])
    for i in range(instance_gt.shape[2]):
        instance_map[instance_gt[..., i] != 0] = i
    return instance_map

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
instance_gt = generate_ground_truth(
    [InstanceGroundTruth(project_to_xy=True)],
    pipeline.dynamics_sim_output,
    slice_proj_kymo_data["n_spatial_values"],
    params.rendering.particle_path_points,
)[GroundTruthFuncCollection.INSTANCE]
instance_map = create_instance_map(instance_gt)


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 15))
axes[0].imshow(mean_proj_kymo, "gray", interpolation="nearest")
axes[0].set_title("Projection method: Mean")
axes[1].imshow(max_proj_kymo, "gray", interpolation="nearest")
axes[1].set_title("Projection method: Max")
axes[2].imshow(slice_proj_kymo, "gray", interpolation="nearest")
axes[2].set_title("Projection method: Centre Slice")
axes[3].imshow(instance_map, "turbo", interpolation="nearest")
axes[3].set_title("GT")